## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [219]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [280]:
from tqdm import tqdm

In [577]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [495]:
import re

# Importar datasets

## NBA regular season stats

Importamos los 21 dataframes, los concatenamos y le asignamos el valor de la temporada en la variable 'season'

In [148]:
data = pd.DataFrame()
for i in range(2001,2022):
    df = pd.read_csv(f'./Data/season_{i}_detailed.csv')
    df['season'] = i
    data = pd.concat([data, df])

In [152]:
data.head()

,date,team,player,role,MP,FG,FGA,FG_PCT,FG3,FG3A,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,season
0,2000-10-31T19:30:00,Charlotte Hornets,P.J. Brown,Starter,38.6333,5,7,0.714,0,0,...,15,16,1,1,2,1,2,14,28.0,2001
1,2000-10-31T19:30:00,Orlando Magic,Darrell Armstrong,Starter,35.5833,4,9,0.444,0,1,...,2,2,3,0,1,3,2,10,10.0,2001
2,2000-10-31T19:30:00,Orlando Magic,Grant Hill,Starter,33.55,3,6,0.500,0,0,...,5,5,10,2,1,5,3,9,5.0,2001
3,2000-10-31T19:30:00,Orlando Magic,John Amaechi,Starter,25.4167,6,10,0.600,0,0,...,3,3,1,1,1,2,4,14,0.0,2001
4,2000-10-31T19:30:00,Orlando Magic,Bo Outlaw,Starter,19.3167,2,4,0.500,0,0,...,2,4,0,1,3,0,3,5,-2.0,2001


## NBA players features

Ahora importamos los 2 dataframes restantes 'players' y 'salaries' y los uniremos en el mismo dataframe bajo el nombre 'database' haciendo join en el campo 'player_id'

In [156]:
players = pd.read_csv('./Data/players.csv')

In [248]:
players.head(3)

,player_id,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,...,draft_pick,draft_round,draft_team,draft_year,height,highSchool,name,position,shoots,weight
0,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
1,abdulza01,"April 7, 1946","Brooklyn, New York",1.2,42.8,NaN,72.8,505,15.1,9.0,...,5th overall,1st round,Cincinnati Royals,1968,6-9,"John Jay in Brooklyn, New York",Zaid Abdul-Aziz,Power Forward and Center,Right,235lb
2,abdulka01,"April 16, 1947","New York, New York",3.6,55.9,5.6,72.1,1560,24.6,24.6,...,1st overall,1st round,Milwaukee Bucks,1969,7-2,"Power Memorial in New York, New York",Kareem Abdul-Jabbar,Center,Right,225lb


In [235]:
players.shape

(4685, 24)

In [233]:
salaries = pd.read_csv('./Data/salaries.csv')

In [249]:
salaries.head(3)

,league,player_id,salary,season,season_end,season_start,team
0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers
1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers
2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics


In [236]:
salaries.shape

(14163, 7)

In [243]:
players.rename(columns = {'_id':'player_id'}, inplace = True)

Creamos nuestro df con los datos de los jugadores 'database'

In [250]:
database = salaries.merge(players, on='player_id', how='left')

Ahora el objetivo es crear una nueva columna en nuestro dataframe 'data' con el nombre correcto, en caso de que éste aparezca en 'database'; con el que tenga un score >85% al compararlo con el método process.extractOne() de la librería fuzzywuzzy o con 'not found' en caso contrario.

In [323]:
# Aplicamos reset_index() para que funcione la iteración de abajo

data = data.reset_index().drop('index', axis=1)

In [332]:
# Aplicamos la iteración hasta el valor nº 463592, que es el primer valor de la temporada 2019

# Hacemos esto porque no tenemos datos en 'database' a partir de esta temporada

for i,c in tqdm(data[0:463592].iterrows()):
    
    # Creamos la variable 'year' para que lo haga con los datos de cada año
    year = c['season']
    
    # Creamos la variable 'names_list' con todos los nombres en nuestra base de datos
    names_list = database[database['season_end'] == year]['name'].unique()
    
    # Si encuentra el nombre dentro del mismo año, que lo ponga en una nueva columna
    if c['player'] in names_list:
        data.loc[i, 'name_ok'] = c['player']
        
    # Si no lo encuentra, que saque el que tenga un parecido mayor al 85%
    elif process.extractOne(c['player'], names_list, scorer=fuzz.token_sort_ratio)[1] >= 85:
        data.loc[i, 'name_ok'] = process.extractOne(c['player'], names_list, scorer=fuzz.token_sort_ratio)[0]
            
    # Si no hay ningún valor con fiabilidad mayor al 85%, que ponga 'not found'
    else:
        data.loc[i, 'name_ok'] = 'not found'
          

463592it [38:46, 199.30it/s]


TypeError: 'NoneType' object is not subscriptable

In [348]:
# Comprobamos que se ha ejecutado correctamente hasta el primer valor del año 2019

data[463591:463593]

,date,team,player,role,MP,FG,FGA,FG_PCT,FG3,FG3A,...,TRB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,season,name_ok
463591,2018-06-08T21:00:00,Golden State Warriors,Andre Iguodala,Reserve,22.3667,4,8,0.500,3,6,...,2,0,2,2,0,3,11,11.0,2018,Andre Iguodala
463592,2018-10-16T20:00:00,Boston Celtics,Gordon Hayward,Starter,24.4,4,12,0.333,1,3,...,5,0,4,0,0,1,10,10.0,2019,NaN


In [345]:
# Sólo un 2% de nuestros datos no han encontrado jugador en nuestro 'database'

len(data[(data['season'] < 2019) & (data['name_ok'] == 'not found')]) / len(data[data['season'] < 2019])

0.019937790125800273

De nuestras entradas entre 2001 y 2018, sólo un 1,99% no han encontrado su jugador equivalente en nuestro 'database'. Vamos a ver cuáles son los jugadores con mayor número de entradas para ver si somos capaces de encontrarlos manualmente en nuestro 'database':

In [360]:
# Vemos cuáles de ellos son los que más intervenciones tienen para intentar buscar su equivalente a mano

data[data['name_ok']=='not found']['player'].value_counts().head(20)

Nenê Hilário            958
Ömer Aşık               489
Didier Ilunga-Mbenga    253
JaKarr Sampson          169
Dario Šarić             163
James Michael McAdoo    131
Brandan Wright          115
Markel Brown            114
Jamaal Tinsley          114
Adreian Payne           107
James Young             101
Kurt Thomas              92
Darrun Hilliard          91
Nate Wolters             84
C.J. Watson              82
Lorenzo Brown            79
Brandon Davies           78
Hollis Thompson          77
Cartier Martin           76
Phil Pressey             75
Name: player, dtype: int64

Aplicamos los cambios a los 3 con más entradas para los cuales tenemos información de salarios en 'database':

In [378]:
for i,c in tqdm(data[0:463592].iterrows()):
    if c['player'] == 'Nenê Hilário':
        data.loc[i, 'name_ok'] = 'Nene'
    elif c['player'] == 'Ömer Aşık':
        data.loc[i, 'name_ok'] = 'Omer Asik'
    elif c['player'] == 'Dario Šarić':
        data.loc[i, 'name_ok'] = 'Dario Saric'

463592it [00:21, 21189.00it/s]


Vemos como este pequeño cambio nos ha hecho pasar de tener un 1,99% de entradas de jugadores sin identificar a tan sólo un 1,64%:

In [380]:
len(data[(data['season'] < 2019) & (data['name_ok'] == 'not found')]) / len(data[data['season'] < 2019])

0.016464908799116464

Ahora cambiamos los nombres de las columnas para que coincidan al hacer el left join de 'database'

In [385]:
data.rename(columns = {'season':'season_end', 'name_ok': 'name'}, inplace = True)

In [386]:
data.head(2)

,date,team,player,role,MP,FG,FGA,FG_PCT,FG3,FG3A,...,TRB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,season_end,name
0,2000-10-31T19:30:00,Charlotte Hornets,P.J. Brown,Starter,38.6333,5,7,0.714,0,0,...,16,1,1,2,1,2,14,28.0,2001,P.J. Brown
1,2000-10-31T19:30:00,Orlando Magic,Darrell Armstrong,Starter,35.5833,4,9,0.444,0,1,...,2,3,0,1,3,2,10,10.0,2001,Darrell Armstrong


In [387]:
database.head(2)

,league,player_id,salary,season,season_end,season_start,team,birthDate,birthPlace,career_AST,...,draft_pick,draft_round,draft_team,draft_year,height,highSchool,name,position,shoots,weight
0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers,"June 24, 1968","Cairo, Egypt",0.3,...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers,"June 24, 1968","Cairo, Egypt",0.3,...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb


## Scrapping

In [431]:
# Intentamos completar la database de los jugadores restantes haciendo web scrapping

# Creamos la tabla auxiliar 'scrap'

scrap = data[data['season_end'] > 2018][['player', 'season_end']].drop_duplicates()

In [435]:
# Habrá que sacar: ['salary', 'birthDate', 'height', 'position', 'shoots', 'weight']

scrap.head(3)

,player,season_end
463592,Gordon Hayward,2019
463593,Marcus Smart,2019
463594,Terry Rozier,2019


In [1054]:
driver = webdriver.Firefox()

In [1055]:
driver.get('https://www.basketball-reference.com/')

In [1056]:
# Aceptamos cookies para no tener que volver a hacerlo al iterar
driver.find_element_by_class_name('qc-cmp2-summary-buttons').find_elements_by_tag_name('button')[2].click()

In [1057]:
# Enviamos cualquier jugador
driver.find_element_by_class_name('ac-wrapper').find_elements_by_tag_name('input')[1].send_keys('Michael Jordan')

In [1058]:
# Enter
driver.find_element_by_class_name('ac-wrapper').find_elements_by_tag_name('input')[1].send_keys(Keys.RETURN)

In [1059]:
try:
    driver.find_element_by_class_name('search-item-name').find_element_by_tag_name('a').click()
except:
    pass

In [1060]:
# Desplegamos una vez los salarios para no tener que volver a hacerlo al iterar
driver.find_element_by_id('meta_more_button').click()

In [1061]:
# Enviamos cualquier otro jugador
driver.find_element_by_class_name('ac-wrapper').find_elements_by_tag_name('input')[1].send_keys('Pau Gasol')

In [1062]:
# Enter
driver.find_element_by_class_name('ac-wrapper').find_elements_by_tag_name('input')[1].send_keys(Keys.RETURN)

In [1063]:
# Volvemos a aceptar
driver.find_element_by_class_name('search-item-name').find_element_by_tag_name('a').click()

In [1064]:
# Ahora ya podemos iterar evitándonos los pasos de aceptar cookies y desplegar los salarios
for i,c in tqdm(scrap.iterrows()):
  
    driver.find_element_by_class_name('ac-wrapper').find_elements_by_tag_name('input')[1].send_keys(f"{c['player']}")
    driver.implicitly_wait(5)
    driver.find_element_by_class_name('ac-wrapper').find_elements_by_tag_name('input')[1].send_keys(Keys.RETURN)
    driver.implicitly_wait(5)
    
    try:
        driver.find_element_by_class_name('search-item-name').find_element_by_tag_name('a').click()
    except:
        pass

    driver.implicitly_wait(5)
    
    # Position
    try:
        scrap.loc[i, 'position'] = re.findall('Position: (\D*) ▪ ', driver.find_element_by_id('meta').text)[0]
    except:
        pass
    
    # Shoots
    try:
        scrap.loc[i, 'shoots'] = re.findall('Shoots: (\D*)\\n', driver.find_element_by_id('meta').text)[0]
    except:
        pass
    
    # Height
    try:
        scrap.loc[i, 'height'] = re.findall('\\n(\d*\-\d*), \d*lb', driver.find_element_by_id('meta').text)[0]
    except:
        pass
    
    # Weight
    try:
        scrap.loc[i, 'weight'] = re.findall('\\n\d*\-\d*, (\d*lb)', driver.find_element_by_id('meta').text)[0]
    except:
        pass
    
    # Birth date
    try:
        scrap.loc[i, 'birthDate'] = driver.find_element_by_id('necro-birth').text
    except:
        pass
    
    # Salary
    if c['season_end'] == 2019:
        try:
            scrap.loc[i, 'salary'] = float(re.findall('2018-19[^\$]*\$(\S*)\\n', driver.find_element_by_id('all_salaries').text)[0].replace(',',''))
        except:
            pass
        
    elif c['season_end'] == 2020:
        try:
            scrap.loc[i, 'salary'] = float(re.findall('2019-20[^\$]*\$(\S*)\\n', driver.find_element_by_id('all_salaries').text)[0].replace(',',''))
        except:
            pass
        
    elif c['season_end'] == 2021:
        try:
            scrap.loc[i, 'salary'] = float(driver.find_element_by_id('div_contract').find_elements_by_tag_name('td')[1].text.replace('$','').replace(',',''))
        except:
            pass
        
    driver.implicitly_wait(5)
    
driver.quit()

248it [19:51,  4.80s/it]


In [1065]:
scrap[-10:]

,player,season_end,position,shoots,height,weight,birthDate,salary
530573,Jeremiah Martin,2021,Point Guard,Left,6-3,185lb,"June 19, 1996",NaN
530887,Gabriel Deck,2021,Power Forward,Right,6-8,232lb,"February 8, 1995",3870000.0
531827,Anderson Varejão,2021,Center and Power Forward,Right,6-10,273lb,"September 28, 1982",1913345.0
531842,Louis King,2021,Small Forward,Right,6-7,205lb,"April 6, 1999",NaN
532216,Khyri Thomas,2021,Shooting Guard,Right,6-3,210lb,"May 8, 1996",744684.0
532770,Cameron Oliver,2021,Small Forward,Right,6-8,239lb,"July 11, 1996",NaN
532857,Jay Scrubb,2021,Shooting Guard,Left,6-5,220lb,"September 1, 2000",NaN
533096,Marcos Louzada Silva,2021,Small Forward,Right,6-5,188lb,"February 7, 1999",123056.0
533213,Udonis Haslem,2021,Power Forward and Center,Right,6-8,235lb,"June 9, 1980",2564753.0
533844,Elijah Bryant,2021,Shooting Guard,Right,6-5,210lb,"April 19, 1995",24611.0


In [1066]:
database.columns

Index(['league', 'player_id', 'salary', 'season', 'season_end', 'season_start',
       'team', 'birthDate', 'birthPlace', 'career_AST', 'career_FG%',
       'career_FG3%', 'career_FT%', 'career_G', 'career_PER', 'career_PTS',
       'career_TRB', 'career_WS', 'career_eFG%', 'college', 'draft_pick',
       'draft_round', 'draft_team', 'draft_year', 'height', 'highSchool',
       'name', 'position', 'shoots', 'weight'],
      dtype='object')

In [1069]:
data.columns

Index(['date', 'team', 'player', 'role', 'MP', 'FG', 'FGA', 'FG_PCT', 'FG3',
       'FG3A', 'FG3_PCT', 'FT', 'FTA', 'FT_PCT', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'season_end', 'name'],
      dtype='object')

### Corregimos Tim Hardaway Jr. por Tim Hardaway

In [1081]:
data[(data['player'] == 'Tim Hardaway') & (data['season_end'] < 2008)].head()

,date,team,player,role,MP,FG,FGA,FG_PCT,FG3,FG3A,...,TRB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,season_end,name
387,2000-11-01T20:00:00,Miami Heat,Tim Hardaway,Starter,25.4167,6,12,0.500,3,6,...,2,7,2,0,1,0,17,32.0,2001,Tim Hardaway
630,2000-11-03T19:30:00,Miami Heat,Tim Hardaway,Starter,38.6333,5,13,0.385,2,7,...,5,5,1,1,4,3,15,1.0,2001,Tim Hardaway
792,2000-11-04T19:30:00,Miami Heat,Tim Hardaway,Starter,38.6333,8,16,0.500,3,7,...,5,11,1,0,2,2,23,0.0,2001,Tim Hardaway
1338,2000-11-08T19:30:00,Miami Heat,Tim Hardaway,Starter,34.5667,3,11,0.273,0,5,...,3,6,0,0,1,1,6,4.0,2001,Tim Hardaway
1719,2000-11-10T21:00:00,Miami Heat,Tim Hardaway,Starter,31.5167,8,14,0.571,1,4,...,2,7,2,0,4,3,17,6.0,2001,Tim Hardaway


In [1082]:
data[(data['player'] == 'Tim Hardaway') & (data['season_end'] > 2008)].head()

,date,team,player,role,MP,FG,FGA,FG_PCT,FG3,FG3A,...,TRB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,season_end,name
329443,2013-10-30T19:30:00,New York Knicks,Tim Hardaway,Reserve,15.25,2,2,1.000,1,1,...,0,2,0,0,0,2,5,0.0,2014,Tim Hardaway Jr.
329681,2013-10-31T20:00:00,New York Knicks,Tim Hardaway,Reserve,26.4333,4,13,0.308,2,6,...,3,0,0,0,0,4,10,7.0,2014,Tim Hardaway Jr.
330249,2013-11-03T19:30:00,New York Knicks,Tim Hardaway,Reserve,24.4,3,12,0.250,0,7,...,1,1,0,0,1,3,6,-7.0,2014,Tim Hardaway Jr.
330437,2013-11-05T19:30:00,New York Knicks,Tim Hardaway,Reserve,11.1833,2,7,0.286,0,4,...,1,1,0,0,0,1,4,-9.0,2014,Tim Hardaway Jr.
330853,2013-11-08T19:00:00,New York Knicks,Tim Hardaway,Reserve,13.2167,2,4,0.500,1,2,...,0,0,0,0,2,1,5,-1.0,2014,Tim Hardaway Jr.


In [1084]:
for i,c in data.iterrows():
    if (c['season_end'] > 2008) and (c['player'] == 'Tim Hardaway'):
        data.loc[i, 'player'] = 'Tim Hardaway Jr.'

In [1044]:
# Corregimos los valores en nuestra tabla de scrapping, que se habían absorbido mal
scrap[scrap['player'] == 'Tim Hardaway']

,player,season_end,position,shoots,height,weight,birthDate,salary
463730,Tim Hardaway,2019,Point Guard,Right,6-0,175lb,"September 1, 1966",NaN
490570,Tim Hardaway,2020,Point Guard,Right,6-0,175lb,"September 1, 1966",NaN
511187,Tim Hardaway,2021,Center and Power Forward,Right,6-10,240lb,"July 20, 1991",10185185.0


In [1088]:
scrap[138:139]

,player,season_end,position,shoots,height,weight,birthDate,salary
463730,Tim Hardaway,2019,Point Guard,Right,6-0,175lb,"September 1, 1966",NaN


In [1117]:
scrap.loc[463730, 'player'] = 'Tim Hardaway Jr.'
scrap.loc[463730, 'position'] = 'Shooting Guard'
scrap.loc[463730, 'shoots'] = 'Right'
scrap.loc[463730, 'height'] = '6-5'
scrap.loc[463730, 'weight'] = '205lb'
scrap.loc[463730, 'birthDate'] = 'March 16, 1992'
scrap.loc[463730, 'salary'] = float(17325000)

In [1096]:
scrap[705:706]

,player,season_end,position,shoots,height,weight,birthDate,salary
490570,Tim Hardaway,2020,Point Guard,Right,6-0,175lb,"September 1, 1966",NaN


In [1118]:
scrap.loc[490570, 'player'] = 'Tim Hardaway Jr.'
scrap.loc[490570, 'position'] = 'Shooting Guard'
scrap.loc[490570, 'shoots'] = 'Right'
scrap.loc[490570, 'height'] = '6-5'
scrap.loc[490570, 'weight'] = '205lb'
scrap.loc[490570, 'birthDate'] = 'March 16, 1992'
scrap.loc[490570, 'salary'] = float(18150000)

In [1098]:
scrap[1335:1336]

,player,season_end,position,shoots,height,weight,birthDate,salary
511187,Tim Hardaway,2021,Center and Power Forward,Right,6-10,240lb,"July 20, 1991",10185185.0


In [1119]:
scrap.loc[511187, 'player'] = 'Tim Hardaway Jr.'
scrap.loc[511187, 'position'] = 'Shooting Guard'
scrap.loc[511187, 'shoots'] = 'Right'
scrap.loc[511187, 'height'] = '6-5'
scrap.loc[511187, 'weight'] = '205lb'
scrap.loc[511187, 'birthDate'] = 'March 16, 1992'
scrap.loc[511187, 'salary'] = float(18975000)

In [1122]:
# Ya estaría corregido
scrap[scrap['player'] == 'Tim Hardaway Jr.']

,player,season_end,position,shoots,height,weight,birthDate,salary
463730,Tim Hardaway Jr.,2019.0,Shooting Guard,Right,6-5,205lb,"March 16, 1992",17325000.0
490570,Tim Hardaway Jr.,2020.0,Shooting Guard,Right,6-5,205lb,"March 16, 1992",18150000.0
511187,Tim Hardaway Jr.,2021.0,Shooting Guard,Right,6-5,205lb,"March 16, 1992",18975000.0


### Exploramos vacíos de la tabla 'scrap' que acabamos de obtener

In [1126]:
scrap.isna().sum()

player          0
season_end      0
position        3
shoots          3
height          3
weight          3
birthDate       3
salary        194
dtype: int64

#### Primero corregimos las 3 que no tienen posición, shoots, height, weight ni birthDate

In [1127]:
scrap[scrap['position'].isna()]

,player,season_end,position,shoots,height,weight,birthDate,salary
463830,Boban Marjanović,2019.0,NaN,NaN,NaN,NaN,NaN,NaN
469888,Danuel House,2019.0,NaN,NaN,NaN,NaN,NaN,NaN
470782,Nenê Hilário,2019.0,NaN,NaN,NaN,NaN,NaN,NaN


In [1128]:
# Antes
scrap[scrap['player'] == 'Boban Marjanović']

,player,season_end,position,shoots,height,weight,birthDate,salary
463830,Boban Marjanović,2019.0,NaN,NaN,NaN,NaN,NaN,NaN
491989,Boban Marjanović,2020.0,Center,Right,7-4,290lb,"August 15, 1988",3500000.0
511304,Boban Marjanović,2021.0,Center,Right,7-4,290lb,"August 15, 1988",3500000.0


In [1133]:
scrap.loc[463830, 'player'] = 'Boban Marjanović'
scrap.loc[463830, 'position'] = 'Center'
scrap.loc[463830, 'shoots'] = 'Right'
scrap.loc[463830, 'height'] = '7-4'
scrap.loc[463830, 'weight'] = '290lb'
scrap.loc[463830, 'birthDate'] = 'August 15, 1988'
scrap.loc[463830, 'salary'] = float(7000000)

In [1134]:
# Después
scrap[scrap['player'] == 'Boban Marjanović']

,player,season_end,position,shoots,height,weight,birthDate,salary
463830,Boban Marjanović,2019.0,Center,Right,7-4,290lb,"August 15, 1988",7000000.0
491989,Boban Marjanović,2020.0,Center,Right,7-4,290lb,"August 15, 1988",3500000.0
511304,Boban Marjanović,2021.0,Center,Right,7-4,290lb,"August 15, 1988",3500000.0


In [1129]:
# Antes
scrap[scrap['player'] == 'Danuel House']

,player,season_end,position,shoots,height,weight,birthDate,salary
469888,Danuel House,2019.0,NaN,NaN,NaN,NaN,NaN,NaN
490693,Danuel House,2020.0,Small Forward and Shooting Guard,Right,6-6,220lb,"June 7, 1993",3540000.0
511515,Danuel House,2021.0,Small Forward and Shooting Guard,Right,6-6,220lb,"June 7, 1993",3717000.0


In [1135]:
scrap.loc[469888, 'player'] = 'Danuel House'
scrap.loc[469888, 'position'] = 'Small Forward and Shooting Guard'
scrap.loc[469888, 'shoots'] = 'Right'
scrap.loc[469888, 'height'] = '6-6'
scrap.loc[469888, 'weight'] = '220lb'
scrap.loc[469888, 'birthDate'] = 'June 7, 1993'
scrap.loc[469888, 'salary'] = float(341830)

In [1136]:
# Después
scrap[scrap['player'] == 'Danuel House']

,player,season_end,position,shoots,height,weight,birthDate,salary
469888,Danuel House,2019.0,Small Forward and Shooting Guard,Right,6-6,220lb,"June 7, 1993",341830.0
490693,Danuel House,2020.0,Small Forward and Shooting Guard,Right,6-6,220lb,"June 7, 1993",3540000.0
511515,Danuel House,2021.0,Small Forward and Shooting Guard,Right,6-6,220lb,"June 7, 1993",3717000.0


In [1130]:
# Antes
scrap[scrap['player'] == 'Nenê Hilário']

,player,season_end,position,shoots,height,weight,birthDate,salary
470782,Nenê Hilário,2019.0,NaN,NaN,NaN,NaN,NaN,NaN


In [1137]:
scrap.loc[470782, 'player'] = 'Nenê Hilário'
scrap.loc[470782, 'position'] = 'Center and Power Forward'
scrap.loc[470782, 'shoots'] = 'Right'
scrap.loc[470782, 'height'] = '6-11'
scrap.loc[470782, 'weight'] = '250lb'
scrap.loc[470782, 'birthDate'] = 'September 13, 1982'
scrap.loc[470782, 'salary'] = float(3651480)

In [1138]:
# Después
scrap[scrap['player'] == 'Nenê Hilário']

,player,season_end,position,shoots,height,weight,birthDate,salary
470782,Nenê Hilário,2019.0,Center and Power Forward,Right,6-11,250lb,"September 13, 1982",3651480.0


#### Ahora vemos qué ha pasado con las que no tienen salario

In [1147]:
scrap[scrap['salary'].isna()]

,player,season_end,position,shoots,height,weight,birthDate,salary
463645,Glenn Robinson,2019.0,Small Forward,Right,6-7,225lb,"January 10, 1973",NaN
463652,Zach Lofton,2019.0,Shooting Guard,Right,6-4,180lb,"November 18, 1992",NaN
463712,Jaren Jackson,2019.0,Shooting Guard,Right,6-4,190lb,"October 27, 1967",NaN
463756,Alex Poythress,2019.0,Power Forward,Right,6-9,235lb,"September 6, 1993",NaN
463766,John Holland,2019.0,Shooting Guard,Right,6-3,180lb,"December 6, 1956",NaN
...,...,...,...,...,...,...,...,...
530261,Charlie Brown,2021.0,Shooting Guard,Right,6-6,199lb,"February 2, 1997",NaN
530573,Jeremiah Martin,2021.0,Point Guard,Left,6-3,185lb,"June 19, 1996",NaN
531842,Louis King,2021.0,Small Forward,Right,6-7,205lb,"April 6, 1999",NaN
532770,Cameron Oliver,2021.0,Small Forward,Right,6-8,239lb,"July 11, 1996",NaN


En muchos casos, nos ha introducido NaN porque el salario aparecía como 'minimum', luego no ha podido pasar el valor a float().

En otros casos, nos ha devuelto un jugador más antiguo (ej: nacido en 1956 jugando en 2019).

Por ahora vamos a rellenar todos los valores con el valor mínimo. Más adelante ya detectaremos el resto de outliers calculando su edad.

In [1140]:
scrap['salary'].min()

4737.0

In [1153]:
scrap['salary'].fillna(float(4737), inplace=True)

Ya no tenemos valores vacíos en nuestra tabla 'scrap'

In [1155]:
scrap.isna().sum()

player        0
season_end    0
position      0
shoots        0
height        0
weight        0
birthDate     0
salary        0
dtype: int64

## Consolidación tablas

Ahora tenemos nuestra tabla original 'data' y además las siguientes tablas auxiliares:

- 'database', con datos desde 2001 hasta 2018
    (esta tabla se une con 'data' a través de los campos 'name' y 'season_end')
    
- 'scrap', con datos desde 2019 hasta 2021
    (esta tabla se une con 'data' a través de los campos 'player' y 'season_end)
    
Debemos unir toda la información en nuestra tabla 'data' y exportarla.

In [1169]:
# Dividimos nuestra tabla en 2
data2001_2018 = data[data['season_end'] <= 2018]
data2019_2021 = data[data['season_end'] > 2018]

In [1171]:
# Hacemos 'merge' de la tabla 'database' con nuestra nueva tabla de 2001 a 2018
data2001_2018 = data2001_2018.merge(database[['salary', 'season_end', 'birthDate', 'height', 'name','position', 'shoots', 'weight']],
           on=['season_end', 'name'], how='left')


In [1177]:
# Hacemos 'merge' de la tabla 'scrap' con nuestra nueva tabla de 2019 a 2021
data2019_2021 = data2019_2021.merge(scrap[['salary', 'season_end', 'birthDate', 'height', 'player','position', 'shoots', 'weight']],
           on=['season_end', 'player'], how='left')


In [1180]:
# Ambas tablas tienen las mismas columnas, por lo que se pueden unir

data2001_2018.columns == data2019_2021.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [1187]:
data2001_2018.shape

(475840, 32)

In [1188]:
data2019_2021.shape

(72224, 32)

In [1186]:
data2001_2018.shape[0] + data2019_2021.shape[0]

548064

In [1189]:
data_nba = pd.concat([data2001_2018, data2019_2021])

In [1190]:
data_nba.shape

(548064, 32)

In [1191]:
# Exportamos los datos para evitar repetir las iteraciones tan largas

data_nba.to_csv('./Data/data_nba.csv')